In [1]:
link1 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap82.txt'
link2 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap113.txt'
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    response = urllib.request.urlretrieve(url,filename)
    file = open(filename,'r')
    return file

In [3]:
file = load_data(link2, 'cap113.txt')
data = file.readlines()

In [4]:
data

[' 50 50 \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 0. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17500. \n',
 ' 5000 17

In [5]:
[locations, customers] = [int(x) for x in data[0].split(' ')[1:-1]]
capacity = []
fixcost = []
for i in range(locations):
    [c, f] = [float(x) for x in data[i+1].split(' ')[1:-1]]
    capacity.append(c)
    fixcost.append(f)
demand = []
varcost = np.zeros((locations,customers))
for j in range(customers):
    demand.append(float(data[j*9 + (1+locations)].split(' ')[1]))
    v = list(np.concatenate([data[j*9 + (1+locations+t)].split(' ')[1:-1] for t in range(1,9)]))
    for i in range(locations):
        varcost[i,j] = float(v[i])/demand[j] #cost of allocating demand of j to warehouse i per unit
        
        
print(locations)
print(customers)
print(capacity)
print(fixcost)
print(demand)
print(varcost)

bigM = min(np.max(capacity), sum(demand))
bigM

50
50
[5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0]
[17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 0.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0]
[146.0, 87.0, 672.0, 1337.0, 31.0, 559.0, 2370.0, 1089.0, 33.0, 32.0, 5495.0, 904.0, 1466.0, 143.0, 615.0, 564.0, 226.0, 3016.0, 253.0, 195.0, 38.

5000.0

In [6]:
#MILP
# Warehouse Location Problem

model = gp.Model("MILP Warehouse Location Problem")

# Create variables
x = {} # transport qty of customer j served by location i
y = {} # binary variables representing decision to open location or not

for i in range(locations):
    for j in range(customers):
        x[i,j] = model.addVar(obj=varcost[i][j])

for i in range(locations):
    y[i] = model.addVar(vtype = GRB.BINARY, obj=fixcost[i])

#Capacity constraint    
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
    
# Demand constraints
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model.addConstr(sum(x[i,j] for i in range(locations)) >= demand[j])
    
#Binding constraint
log_constr = {}
for i in range(locations):
    log_constr[i] = model.addConstr(sum(x[i,j] for j in range(customers)) <= y[i]* bigM)


model.optimize()


print("Objective: " + str(model.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x+1) for x in range(locations)]
columns = ['Customer ' + str(x+1) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns+['y'])

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x[i,j].x
    solution.iloc[i,customers] = y[i].x


print(solution)

#print('\nRunning time: ', model.Runtime)

#model.dispose()

Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 150 rows, 2550 columns and 7550 nonzeros
Model fingerprint: 0xfaf7a501
Variable types: 2500 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 1e+04]
Found heuristic solution: objective 2762259.6250
Presolve removed 50 rows and 1 columns
Presolve time: 0.02s
Presolved: 100 rows, 2549 columns, 5049 nonzeros
Variable types: 2500 continuous, 49 integer (49 binary)

Root relaxation: objective 8.430595e+05, 113 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 843059.512    0   44 2762259.63 843059.512  69.5%     -    0s
H    0     0                    